In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np

from utilities import remove_cols

2024-02-11 15:37:54.261577: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 15:37:54.289836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))

For tuning we will use a random sample of the same size that our samples will eventually be. This involves allocating 600 words for test, and the rest for train - but not using our pre-allocated samples for the purpose.

In [3]:
np.random.seed(982)

train_n = X.shape[0]
test_n = train_n-600

sample = np.full(train_n, False, dtype=bool)

indices = np.random.choice(train_n, test_n, replace=False)

# Set chosen indices to True
sample[indices] = True

In [134]:
hidden = 100
seed = 387
import time

def learner(seed, learning_rate, hidden):
    np.random.seed(seed)

    optimizer = Adam(learning_rate=learning_rate)

    model = Sequential()
    model.add(Dense(hidden, input_shape=(X.shape[1],)))
    model.add(Dense(Y.shape[1], activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy', 'mse'])
    
    return model

with open('outputs/tune_1.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{}\n".format("learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.075, .1, .125, .15, .175, .2, .225, .25]: 
        for batch_size in [32, 64, 128, 256]:
            for epochs in [100, 150, 200, 250, 500, 1000]:
                model = learner(seed, learning_rate, hidden)
                
                start_time = time.time()


                model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                end_time = time.time()
                runtime = end_time - start_time

                loss_train, accuracy_train, __, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) # loss is duplicated so the 2nd value is set to __
                loss_test, accuracy_test, __, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) # loss is duplicated so the 2nd value is set to __

                f.write("{},{},{},{},{},{},{},{},{},{}\n".format(learning_rate,
                                                batch_size,
                                                epochs,
                                                loss_train,
                                                accuracy_train,
                                                mse_train,
                                                loss_test,
                                                accuracy_test,
                                                mse_test,
                                                runtime))
f.close()